## Import

In [1]:
import importlib
import helpers
importlib.reload(helpers)

import pickle
import regex
import sklearn_crfsuite

from helpers import label_text
from sklearn_crfsuite import metrics

## Load Data

In [2]:
au_texts = pickle.load(open(r'/mnt/Data/wiki/countries/australia/data/extracted_texts.pkl', 'rb'))

In [3]:
#list(au_texts.keys())[:20]

In [4]:
domains = {
    'www.trimec.com.au': {
        'name': 'Assa Abloy',
        'address': '235 Huntingdale Road, Oakleigh, VIC 3166, Melbourne, Australia',
        'reg_id': '84 050 103 896',
    },
    'www.beangrowers.com.au': {
        'name': 'Bean Growers Australia',
        'address': '82-86 River Road, Kingaroy, Queensland, Australia 4610',
        'reg_id': '52 092 429 984',
    },
    'www.netforge.com.au': {
        'name': 'Netforge',
        'address': '84 Halifax Street, Adelaide 5000, SA.',
        'reg_id': '30 769 018 561',
    },
    'www.smartemp.com.au': {
        'name': 'Smartair Diffusion',
        'address': 'Jabez Street, Marrickville, NSW, Australia',
        'reg_id': '32 098 233 928',
    },
    'www.syncmarketing.com.au': {
        'name': 'Sync Marketing',
        'address': 'Parramatta, NSW, Australia',
        'reg_id': '35 6060715 74',
    },
}

In [5]:
labeled_texts = []
for domain in domains:
    domain_labels = label_text(au_texts[domain],
                               domains[domain]['name'],
                               domains[domain]['address'],
                               domains[domain]['reg_id'])
    labeled_texts.append(domain_labels)

[\.,\-\\\/\'\"]?Huntingdale\s?[\.,\-\\\/\'\"]?\s[\.,\-\\\/\'\"]?Road\s?[\.,\-\\\/\'\"]?
[\.,\-\\\/\'\"]?Oakleigh\s?[\.,\-\\\/\'\"]?
[\.,\-\\\/\'\"]?VIC\s?[\.,\-\\\/\'\"]?
[\.,\-\\\/\'\"]?Melbourne\s?[\.,\-\\\/\'\"]?
[\.,\-\\\/\'\"]?Australia\s?[\.,\-\\\/\'\"]?
[\.,\-\\\/\'\"]?assa\s?[\.,\-\\\/\'\"]?\s[\.,\-\\\/\'\"]?abloy\s?[\.,\-\\\/\'\"]?
[\.,\-\\\/\'\"]?84050103896\s?[\.,\-\\\/\'\"]?
[\.,\-\\\/\'\"]?River\s?[\.,\-\\\/\'\"]?\s[\.,\-\\\/\'\"]?Road\s?[\.,\-\\\/\'\"]?
[\.,\-\\\/\'\"]?Kingaroy\s?[\.,\-\\\/\'\"]?
[\.,\-\\\/\'\"]?Queensland\s?[\.,\-\\\/\'\"]?
[\.,\-\\\/\'\"]?Australia\s?[\.,\-\\\/\'\"]?
[\.,\-\\\/\'\"]?bean\s?[\.,\-\\\/\'\"]?\s[\.,\-\\\/\'\"]?growers\s?[\.,\-\\\/\'\"]?\s[\.,\-\\\/\'\"]?australia\s?[\.,\-\\\/\'\"]?
[\.,\-\\\/\'\"]?52092429984\s?[\.,\-\\\/\'\"]?
[\.,\-\\\/\'\"]?Halifax\s?[\.,\-\\\/\'\"]?\s[\.,\-\\\/\'\"]?Street\s?[\.,\-\\\/\'\"]?
[\.,\-\\\/\'\"]?Adelaide\s?[\.,\-\\\/\'\"]?
[\.,\-\\\/\'\"]?SA\s?[\.,\-\\\/\'\"]?
[\.,\-\\\/\'\"]?netforge\s?[\.,\-\\\/\'\"]?
[\.,

In [9]:
labeled_texts[3]

[('Contact', 'O'),
 ('|', 'O'),
 ('Smartair', 'B-org'),
 ('Diffusion', 'I-org'),
 ('^', 'O'),
 ('Smartair', 'B-org'),
 ('Diffusion', 'I-org'),
 ('contact', 'O'),
 ('us', 'O'),
 ('page', 'O'),
 ('^', 'O'),
 ('Products', 'O'),
 ('^', 'O'),
 ('About', 'O'),
 ('^', 'O'),
 ('Contact', 'O'),
 ('^', 'O'),
 ('Home', 'O'),
 ('^', 'O'),
 ('Contact', 'O'),
 ('^', 'O'),
 ('Contact', 'O'),
 ('^', 'O'),
 ('SUPPORT', 'O'),
 ('CENTRE', 'O'),
 ('^', 'O'),
 ('Unit', 'O'),
 ('2', 'geoDigit'),
 (',', 'O'),
 ('17', 'geoDigit'),
 ('Jabez', 'B-geo'),
 ('Street', 'I-geo'),
 (',', 'O'),
 ('^', 'O'),
 ('Marrickville', 'B-geo'),
 (',', 'O'),
 ('NSW', 'B-geo'),
 ('2204', 'geoDigit'),
 (',', 'O'),
 ('^', 'O'),
 ('Australia', 'B-geo'),
 ('^', 'O'),
 ('P', 'O'),
 ('^', 'O'),
 ('+', 'O'),
 ('611300572689', 'O'),
 ('^', 'O'),
 ('F', 'O'),
 ('^', 'O'),
 ('+', 'O'),
 ('61282132764', 'O'),
 ('^', 'O'),
 ('E', 'O'),
 ('^', 'O'),
 ('info', 'O'),
 ('@', 'O'),
 ('smartair', 'O'),
 ('.', 'O'),
 ('asia', 'O'),
 ('^', 'O'),
 ('

In [10]:
# pickle.dump(labeled_texts, open('labeled_texts_sample.pkl', 'wb'))

In [11]:
import json
with open('labeled_texts_sample.txt', 'w') as outfile:
    json.dump(labeled_texts, outfile)

In [1]:
from labeled_texts_sample import texts

In [2]:
len(texts)

5

## Feature extraction functions

In [34]:
def word2features(sent, i):
    word = sent[i][0]
    
    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        #'word[-3:]': word[-3:],
        #'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'word_length': len(word),
    }
    if i > 1:
        word1 = sent[i-1][0]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
        })   
        
        word2 = sent[i-2][0]
        features.update({
            '-2:word.lower()': word2.lower(),
            '-2:word.istitle()': word2.istitle(),
            '-2:word.isupper()': word2.isupper(),
        })  
        
    elif i == 1:
        word1 = sent[i-1][0]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
        })   
    else:
        features['BOS'] = True
        
    if i < len(sent)-1:
        word1 = sent[i+1][0]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
        })
    else:
        features['EOS'] = True
                
    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, label in sent]

def sent2tokens(sent):
    return [token for token, label in sent]

In [51]:
sent2features(labeled_texts[0])[3]

{'+1:word.istitle()': False,
 '+1:word.isupper()': False,
 '+1:word.lower()': '^',
 '-1:word.istitle()': False,
 '-1:word.isupper()': False,
 '-1:word.lower()': '-',
 '-2:word.istitle()': False,
 '-2:word.isupper()': False,
 '-2:word.lower()': 'notice',
 'bias': 1.0,
 'word.isdigit()': False,
 'word.istitle()': True,
 'word.isupper()': False,
 'word.lower()': 'trimec',
 'word_length': 6}

## Train, test split

In [39]:
X_train = [sent2features(s) for s in labeled_texts[:3]]
y_train = [sent2labels(s) for s in labeled_texts[:3]]

X_test = [sent2features(s) for s in labeled_texts[3:]]
y_test = [sent2labels(s) for s in labeled_texts[3:]]

## Train

In [47]:
%%time
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs', 
    c1=0.1, 
    c2=0.1, 
    max_iterations=200, 
    all_possible_transitions=True,
    verbose=True
)
crf.fit(X_train, y_train)

loading training data to CRFsuite: 100%|██████████| 3/3 [00:00<00:00, 12.33it/s]



Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 7826
Seconds required: 0.039

L-BFGS optimization
c1: 0.100000
c2: 0.100000
num_memories: 6
max_iterations: 200
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

Iter 1   time=0.02  loss=12904.42 active=7822  feature_norm=1.00
Iter 2   time=0.01  loss=11910.37 active=7760  feature_norm=1.05
Iter 3   time=0.01  loss=8560.64  active=6820  feature_norm=1.56
Iter 4   time=0.01  loss=6741.22  active=6571  feature_norm=1.83
Iter 5   time=0.05  loss=5326.67  active=6519  feature_norm=1.96
Iter 6   time=0.01  loss=4861.35  active=6545  feature_norm=2.54
Iter 7   time=0.03  loss=4387.38  active=6405  feature_norm=2.61
Iter 8   time=0.01  loss=4103.06  active=6422  feature_norm=2.84
Iter 9   time=0.01  loss=3516.19  active=6387  feature_norm=3.01
Iter 10  time=0

## Evaluation

In [48]:
labels = list(crf.classes_)
labels.remove('O')
labels

['B-Comp', 'I-Comp', 'B-BizWord', 'B-Geo', 'GeoDigit', 'I-Geo', 'B-Reg']

In [49]:
y_pred = crf.predict(X_test)
metrics.flat_f1_score(y_test, y_pred, 
                      average='weighted', labels=labels)

/home/vitaliy/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


0.12627915071353984

In [43]:
sorted_labels = sorted(
    labels, 
    key=lambda name: (name[1:], name[0])
)
print(metrics.flat_classification_report(
    y_test, y_pred, labels=sorted_labels, digits=3
))

             precision    recall  f1-score   support

  B-BizWord      0.000     0.000     0.000         5
     B-Comp      0.000     0.000     0.000        54
     I-Comp      0.000     0.000     0.000        54
      B-Geo      0.900     0.220     0.353        41
      I-Geo      0.000     0.000     0.000         1
      B-Reg      0.000     0.000     0.000         7
   GeoDigit      1.000     0.182     0.308        33

avg / total      0.358     0.077     0.126       195



/home/vitaliy/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [44]:
for i in range(len(labeled_texts[3])):
    print(labeled_texts[3][i], ' -- ', y_pred[0][i])

('Contact', 'O')  --  O
('|', 'O')  --  O
('Smartair', 'B-Comp')  --  O
('Diffusion', 'I-Comp')  --  O
('^', 'O')  --  O
('Smartair', 'B-Comp')  --  O
('Diffusion', 'I-Comp')  --  O
('contact', 'O')  --  O
('us', 'O')  --  O
('page', 'O')  --  O
('^', 'O')  --  O
('Products', 'O')  --  O
('^', 'O')  --  O
('About', 'O')  --  O
('^', 'O')  --  O
('Contact', 'O')  --  O
('^', 'O')  --  O
('Home', 'O')  --  O
('^', 'O')  --  O
('Contact', 'O')  --  O
('^', 'O')  --  O
('Contact', 'O')  --  O
('^', 'O')  --  O
('SUPPORT', 'O')  --  O
('CENTRE', 'O')  --  O
('^', 'O')  --  O
('Unit', 'O')  --  O
('2', 'GeoDigit')  --  O
(',', 'O')  --  O
('17', 'GeoDigit')  --  O
('Jabez', 'B-Geo')  --  O
('Street', 'I-Geo')  --  O
(',', 'O')  --  O
('^', 'O')  --  O
('Marrickville', 'B-Geo')  --  O
(',', 'O')  --  O
('NSW', 'B-Geo')  --  O
('2204', 'GeoDigit')  --  O
(',', 'O')  --  O
('^', 'O')  --  O
('Australia', 'B-Geo')  --  B-Geo
('^', 'O')  --  O
('P', 'O')  --  O
('^', 'O')  --  O
('+', 'O')  --  O

## Classifier performance

In [45]:
from collections import Counter

def print_transitions(trans_features):
    for (label_from, label_to), weight in trans_features:
        print("%-6s -> %-7s %0.6f" % (label_from, label_to, weight))

print("Top likely transitions:")
print_transitions(Counter(crf.transition_features_).most_common(20))

print("\nTop unlikely transitions:")
print_transitions(Counter(crf.transition_features_).most_common()[-20:])

Top likely transitions:
O      -> O       3.682329
B-Comp -> I-Comp  2.991069
GeoDigit -> B-Geo   1.930182
I-Comp -> I-Comp  1.596167
B-Geo  -> I-Geo   1.525787
B-Geo  -> GeoDigit 1.204905
B-BizWord -> B-BizWord 1.077347
O      -> B-Comp  1.003471
B-Geo  -> B-Geo   0.735615
I-Comp -> B-BizWord 0.412956
O      -> B-Geo   0.297363
B-BizWord -> O       0.139698
I-Geo  -> O       0.010893
I-Comp -> B-Comp  -0.000093
GeoDigit -> B-Comp  -0.000190
B-Comp -> B-BizWord -0.000622
B-Comp -> B-Geo   -0.004707
B-BizWord -> I-Comp  -0.011741
O      -> GeoDigit -0.071418
B-Reg  -> O       -0.104135

Top unlikely transitions:
B-Comp -> B-BizWord -0.000622
B-Comp -> B-Geo   -0.004707
B-BizWord -> I-Comp  -0.011741
O      -> GeoDigit -0.071418
B-Reg  -> O       -0.104135
B-BizWord -> B-Geo   -0.128847
I-Comp -> GeoDigit -0.228721
O      -> B-BizWord -0.269364
I-Comp -> O       -0.371403
B-Comp -> B-Comp  -0.386113
O      -> B-Reg   -0.392845
B-Reg  -> B-Geo   -0.429902
B-Geo  -> O       -0.431287
B-Geo

In [46]:
def print_state_features(state_features):
    for (attr, label), weight in state_features:
        print("%0.6f %-8s %s" % (weight, label, attr))    

print("Top positive:")
print_state_features(Counter(crf.state_features_).most_common(30))

print("\nTop negative:")
print_state_features(Counter(crf.state_features_).most_common()[-30:])

Top positive:
8.049438 B-Comp   word.lower():netforge
7.909732 B-Geo    word.lower():kingaroy
7.637247 B-BizWord word.lower():limited
4.993854 B-Geo    word.lower():adelaide
4.342166 B-Geo    word.lower():australia
4.083172 B-BizWord word.lower():group
3.809544 B-Geo    word.lower():queensland
3.791430 GeoDigit word.isdigit()
3.349543 B-Reg    word.lower():52092429984
3.273950 I-Comp   word.lower():abloy
3.273950 I-Comp   -1:word.lower():assa
3.221549 O        bias
3.052044 B-Geo    word.lower():sa
2.740114 I-Comp   +1:word.lower():australia
2.637882 B-Comp   word.lower():assa
2.637882 B-Comp   +1:word.lower():abloy
2.503135 I-Comp   word.lower():australia
2.462946 O        -2:word.lower():netforge
2.417465 GeoDigit word.lower():328
2.363072 I-Comp   -1:word.lower():growers
2.310945 I-Geo    word.lower():road
2.264933 I-Comp   -2:word.lower():bean
2.234139 O        word.lower():,
2.156726 B-BizWord word.lower():ltd
2.156726 B-BizWord -1:word.lower():pty
2.082787 B-Geo    +1:word.lower(